### Setup

In [1]:
%%capture
%load_ext kedro.ipython

In [2]:
%%capture
import logging
import os
import pickle

import pandas as pd
from dotenv import load_dotenv
from neo4j import GraphDatabase
from neo4j.exceptions import DriverError, Neo4jError
from sklearn.metrics import completeness_score, homogeneity_score, v_measure_score

load_dotenv(r"..\conf\local\.env")

In [3]:
# Parameters
MODEL_NAME: str = "multi-qa-mpnet-base-cos-v1"
CONTRIBUTOR: str = "Health Promotion Board"

# specify content_category. input 'all' if running across all categories
CONTENT_CATEGORY: str = "live-healthy-articles"

# adjust accordingly
THRESHOLD: float = 0.7

In [4]:
INPUT_GROUNDTRUTH_PATH = os.path.join(
    "..",
    "data",
    "01_raw",
    "Synapxe Content Prioritisation - Live Healthy_020724.xlsx",
)

DATA_FOLDER_PATH = os.path.join(
    "..",
    "data",
    "07_model_output",
    f"{CONTENT_CATEGORY}",
)

INPUT_EMBEDDING_NEO4J_PATH = os.path.join(
    DATA_FOLDER_PATH,
    f"{CONTENT_CATEGORY}_{MODEL_NAME}_embeddings_neo4j.pkl",
)

# Cluster metrics output
OUTPUT_CLUSTER_METRICS_PATH = os.path.join(
    "..",
    "data",
    "07_model_output",
    f"{CONTENT_CATEGORY}",
    f"{CONTENT_CATEGORY}_compiled_model_variation_metrics.csv",
)

## Load files

In [5]:
# Load merged_df data
merged_data_df = catalog.load("merged_data")  # noqa

# load ground truth data
ground_truth = pd.read_excel(INPUT_GROUNDTRUTH_PATH, sheet_name=2)
ground_truth = ground_truth[ground_truth["Owner"].str.contains(CONTRIBUTOR)]
ground_truth = ground_truth[["Page Title", "Combine Group ID", "URL"]]
ground_truth = ground_truth[ground_truth["Combine Group ID"].notna()]

# Extract id from merged_data_df to ground truth
ground_truth = pd.merge(
    ground_truth, merged_data_df, how="inner", left_on="URL", right_on="full_url"
)
ground_truth = ground_truth[["id", "Page Title", "URL", "Combine Group ID"]]
ground_truth.rename(columns={"Combine Group ID": "ground_truth_cluster"}, inplace=True)

print(ground_truth.shape)
ground_truth.head(2)

[07/01/24 19:57:46] INFO     Loading data from merged_data (ParquetDataset)...                  ]8;id=891211;file://c:\Users\Joycelyn\anaconda3\envs\genai-hh\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=913889;file://c:\Users\Joycelyn\anaconda3\envs\genai-hh\Lib\site-packages\kedro\io\data_catalog.py#508\508]8;;\

(184, 4)


,id,Page Title,URL,ground_truth_cluster
0,1442828,Getting ready for solids,https://www.healthhub.sg/live-healthy/baby-get...,1.0
1,1445136,Getting Your Baby Started on Solids,https://www.healthhub.sg/live-healthy/getting-...,1.0


In [6]:
# load embeddings file
with open(INPUT_EMBEDDING_NEO4J_PATH, "rb") as f:
    articles = pickle.load(f)

# merge with ground truth
articles_df = pd.merge(
    articles,
    ground_truth,
    how="inner",
    left_on="id",
    right_on="id",
)

vector_columns = [col for col in articles_df.columns if "vector" in col]
for col in vector_columns:
    articles_df[col] = articles_df[col].apply(lambda x: x.tolist())

print(articles_df.shape)
articles_df.head(2)

(176, 11)


,id,title,content,meta_description,vector_title,vector_article_category_names,vector_category_description,vector_extracted_content_body,Page Title,URL,ground_truth_cluster
0,1443987,All You Need to Know About Childhood Immunisat...,Every child in Singapore is vaccinated accordi...,"To prevent diseases such as measles and mumps,...","[0.026093585416674614, -0.08094950020313263, 0...","[0.06738588958978653, 0.01577157713472843, -0....","[-0.030145417898893356, -0.01659725420176983, ...","[0.032343365252017975, -0.04923023656010628, -...",All You Need to Know About Childhood Immunisat...,https://www.healthhub.sg/live-healthy/all-you-...,16.0
1,1442828,Getting ready for solids,Weaning Tips\nThe process of switching an infa...,You have breastfed your baby for 6 months and ...,"[0.059254396706819534, 0.08260774612426758, -0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.032028209418058395, 0.01828153245151043, 0....","[0.02785884588956833, 0.027463257312774658, 0....",Getting ready for solids,https://www.healthhub.sg/live-healthy/baby-get...,1.0


## Connect to neo4j

In [7]:
uri = "neo4j://localhost:7687"
username = os.getenv("neo4j_username")
password = os.getenv("neo4j_password")
# driver = GraphDatabase.driver(uri, auth=(username, password))

NEO4J = {
    "uri": uri,
    "auth": (username, password),
    "database": CONTENT_CATEGORY,  # create this database in neo4j first
}

# Test connection
with GraphDatabase.driver(**NEO4J) as driver:
    try:
        driver.verify_connectivity()
        print("Connection estabilished.")
    except (DriverError, Neo4jError) as exception:
        print(exception)

Connection estabilished.


## Clustering

In [8]:
logging.basicConfig(level=logging.INFO)


def clear_db(tx):
    logging.info("Clearing database")
    tx.run("MATCH (n) DETACH DELETE n")


def create_graph_nodes(tx, doc):
    # logging.info("Create nodes")
    tx.run(
        """
    CREATE (d:Article {
        id: $id,
        title: $title,
        content: $content,
        meta_desc: $meta_description,
        vector_body: $vector_body,
        vector_title: $vector_title,
        vector_category: $vector_category,
        vector_desc: $vector_desc,
        ground_truth: $ground_truth
    })""",
        id=doc["id"],
        title=doc["title"],
        content=doc["content"],
        meta_description=doc["meta_description"],
        vector_title=doc["vector_title"],
        vector_category=doc["vector_article_category_names"],
        vector_desc=doc["vector_category_description"],
        vector_body=doc["vector_extracted_content_body"],
        ground_truth=doc["ground_truth_cluster"],
    )


def create_sim_edges(tx, threshold):
    logging.info("Create edges")
    tx.run(
        """
    MATCH (a:Article), (b:Article)
    WHERE a.id < b.id
    WITH a, b, gds.similarity.cosine(a.vector_body, b.vector_body) AS similarity
    WHERE similarity > $threshold
    CREATE (a)-[:SIMILAR {similarity: similarity}]->(b)
    """,
        threshold=threshold,
    )


def drop_graph_projection(tx):
    result = tx.run(
        """
    CALL gds.graph.exists('articleGraph')
    YIELD exists
    RETURN exists
    """
    )
    if result.single()["exists"]:
        tx.run("CALL gds.graph.drop('articleGraph')")


def create_graph_proj(tx):
    # logging.info("Create projection")
    tx.run(
        """
           CALL gds.graph.project(
            'articleGraph',
            'Article',
            {
                SIMILAR: {
                    properties: 'similarity'
                }
            }
           )
    """
    )


def detect_community(tx):
    # logging.info("Detect community")
    tx.run(
        """
        CALL gds.louvain.write(
        'articleGraph',
        {
            writeProperty: 'community'
        }
        )
    """
    )


def return_community(tx):
    query = """
        MATCH (a:Article)
        RETURN a.community AS cluster, collect(a.title) AS articles
        ORDER BY cluster
        """
    result = tx.run(query)
    return [record for record in result]


def return_pred_cluster(tx):
    query = """
        MATCH (a:Article)
        RETURN a.id, a.title, a.community AS cluster
        ORDER BY a.community
        """
    result = tx.run(query)
    return [record for record in result]


def count_articles(tx):
    query = """
        MATCH (a:Article)
        RETURN a.community AS cluster, count(a) AS articleCount
        ORDER BY cluster
        """
    result = tx.run(query)
    return [record for record in result]


def get_cluster_min_max(neo4j_clusters_df):
    value_counts = neo4j_clusters_df["number_articles"].value_counts()
    filtered_counts = value_counts[value_counts > 1]
    max = filtered_counts.max()
    min = filtered_counts.min()
    return min, max

In [9]:
documents = articles_df.to_dict(orient="records")
documents[0].keys()

dict_keys(['id', 'title', 'content', 'meta_description', 'vector_title', 'vector_article_category_names', 'vector_category_description', 'vector_extracted_content_body', 'Page Title', 'URL', 'ground_truth_cluster'])

In [10]:
with GraphDatabase.driver(**NEO4J) as driver:
    with driver.session() as session:
        session.execute_write(clear_db)  # Clear the database
        for doc in documents:
            session.execute_write(create_graph_nodes, doc)
        session.execute_write(create_sim_edges, THRESHOLD)
        session.execute_write(drop_graph_projection)
        session.execute_write(create_graph_proj)
        session.execute_write(detect_community)
        records = session.execute_read(return_community)
        pred_cluster = session.execute_read(return_pred_cluster)
        articles_count = session.execute_read(count_articles)

[07/01/24 19:58:04] WARNING  Received notification from DBMS server: {severity: WARNING} {code:       ]8;id=977083;file://c:\Users\Joycelyn\anaconda3\envs\genai-hh\Lib\site-packages\neo4j\_sync\work\result.py\result.py]8;;\:]8;id=996571;file://c:\Users\Joycelyn\anaconda3\envs\genai-hh\Lib\site-packages\neo4j\_sync\work\result.py#321\321]8;;\
                             Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category:                
                             DEPRECATION} {title: This feature is deprecated and will be removed in                
                             future versions.} {description: The query used a deprecated field from a              
                             procedure. ('schema' returned by 'gds.graph.drop' is deprecated.)}                    
                             {position: line: 1, column: 1, offset: 0} for query: "CALL                            
                             gds.graph.drop('articleGraph')"                                                       

In [11]:
pred_cluster_df = pd.DataFrame(pred_cluster, columns=["id", "title", "pred_cluster"])

cluster_article_count = pd.DataFrame(
    articles_count, columns=["pred_cluster_number", "article_count"]
)

In [12]:
results_df = pd.merge(
    articles_df, pred_cluster_df, how="inner", left_on="id", right_on="id"
)

results_df = results_df[
    ["id", "Page Title", "URL", "ground_truth_cluster", "pred_cluster"]
]
results_df["ground_truth_cluster"] = results_df["ground_truth_cluster"].astype(int)

results_df.head(2)

,id,Page Title,URL,ground_truth_cluster,pred_cluster
0,1443987,All You Need to Know About Childhood Immunisat...,https://www.healthhub.sg/live-healthy/all-you-...,16,79
1,1442828,Getting ready for solids,https://www.healthhub.sg/live-healthy/baby-get...,1,81


## Cluster metrics

In [13]:
def get_exact_match(results_df):
    pred_cluster_labels = results_df.groupby("pred_cluster")["id"].apply(set).to_list()
    ground_cluster_labels = (
        results_df.groupby("ground_truth_cluster")["id"].apply(set).to_list()
    )
    complete_match = [s for s in pred_cluster_labels if s in ground_cluster_labels]

    return len(complete_match)


def fill_single(series):
    max_val = series.max()
    fill_in_val = max_val
    filled_series = series.copy()
    for idx in series[series.isna()].index:
        filled_series.at[idx] = fill_in_val + 1
        fill_in_val += 1
    return filled_series.to_list()


def compute_vmeasure(results_df):
    ground_truth_labels = fill_single(results_df["ground_truth_cluster"])
    predicted_labels = fill_single(results_df["pred_cluster"])
    homogeneity = homogeneity_score(ground_truth_labels, predicted_labels)
    completeness = completeness_score(ground_truth_labels, predicted_labels)
    v_measure = v_measure_score(ground_truth_labels, predicted_labels)

    return homogeneity, completeness, v_measure

In [14]:
min_count = cluster_article_count[cluster_article_count["article_count"] > 1][
    "article_count"
].min()
max_count = cluster_article_count["article_count"].max()
num_clusters = len(cluster_article_count)
unclustered_count = (cluster_article_count["article_count"] == 1).sum()

exact_match = get_exact_match(results_df)
homogeneity, completeness, v_measure = compute_vmeasure(results_df)

data = pd.DataFrame(
    {
        "Model": [MODEL_NAME],
        "Threshold": [THRESHOLD],
        "Exact cluster match": [exact_match],
        "Homogeneity": [round(homogeneity, 4)],
        "Completeness": [round(completeness, 4)],
        "V-measure": [round(v_measure, 4)],
        "Number of clusters": [num_clusters],
        "Min cluster size": [min_count],
        "Max cluster size": [max_count],
        "Number of articles not clustered": [unclustered_count],
    }
)

In [15]:
if os.path.exists(OUTPUT_CLUSTER_METRICS_PATH):
    metrics_df = pd.read_csv(OUTPUT_CLUSTER_METRICS_PATH, index_col=0)
else:
    metrics_df = pd.DataFrame()

metrics_df = pd.concat([metrics_df, data], axis=1)
metrics_df.to_csv(OUTPUT_CLUSTER_METRICS_PATH, index=False)
metrics_df

,Model,Threshold,Exact cluster match,Homogeneity,Completeness,V-measure,Number of clusters,Min cluster size,Max cluster size,Number of articles not clustered
0,multi-qa-mpnet-base-cos-v1,0.7,14,0.796,0.8995,0.8446,51,2,23,23
